In [1]:
%defaultDatasource jdbc:h2:mem:db

In [98]:
DROP Table IF EXISTS DADOSMALARIA;
DROP Table IF EXISTS DADOSSUICIDIO;
DROP Table IF EXISTS TOPIDHSUICIDE;
DROP Table IF EXISTS MOZAMBIQUEDATA;
DROP Table IF EXISTS TOP20IDH;
DROP Table IF EXISTS TOPIDHSUICIDE;
DROP Table IF EXISTS TOPIDH;
DROP Table IF EXISTS VARIACAOIDH;
DROP Table IF EXISTS IDH2016;
DROP Table IF EXISTS IDH1990;
DROP Table IF EXISTS VINTEIDH;
DROP Table IF EXISTS TuberculoseBrazil;
DROP Table IF EXISTS Countries;
DROP Table IF EXISTS IDH;
DROP Table IF EXISTS Mortality;
DROP Table IF EXISTS Sanitation;

In [99]:
CREATE TABLE Countries (
  Country VARCHAR(50),
  Region VARCHAR(50),
  Climate VARCHAR(5),
  PRIMARY KEY(country)
) AS SELECT
    country,
    Region,
    Climate
FROM CSVREAD('../data/processed/countries_data.csv');

In [100]:
CREATE TABLE Sanitation (
  REF_AREA VARCHAR(3),
  Country VARCHAR(50) NOT NULL,
  INDICATOR VARCHAR(50) NOT NULL,
  ServiceType VARCHAR(50),
  year INTEGER NOT NULL,
  UnitOfMeasure VARCHAR(3),
  OBS_VALUE FLOAT,
  FOREIGN KEY(Country)
    REFERENCES Countries(Country)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    REF_AREA,
    Country,
    INDICATOR,
    ServiceType,
    year,
    UnitOfMeasure,
    OBS_VALUE
FROM CSVREAD('../data/processed/sanitation_data.csv');

In [101]:
CREATE TABLE IDH (
  country VARCHAR(50) NOT NULL,
  HDIRank INTEGER,
  year INTEGER NOT NULL,
  HDI_VALUE FLOAT,
  FOREIGN KEY(country)
    REFERENCES Countries(country)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    country,
    HDIRank,
    year,
    HDI_VALUE
FROM CSVREAD('../data/processed/IDH_data.csv');

In [102]:
CREATE TABLE Mortality(
    country VARCHAR(50) NOT NULL,
    year INTEGER NOT NULL,
    Tuberculosis FLOAT,
    Suicide FLOAT,
    Drug_disorders FLOAT,
    Cardiovascular_diseases FLOAT, 
    Cancers FLOAT,
    Respiratory_diseases FLOAT, 
    Diabetes FLOAT,
    Dementia FLOAT,
    Lower_respiratory_infections FLOAT,
    Neonatal_deaths FLOAT,
    Diarrheal_diseases FLOAT,
    Road_accidents FLOAT,
    Liver_disease FLOAT,
    Kidney_disease FLOAT,
    Digestive_diseases FLOAT,
    Malaria FLOAT,
    Homicide FLOAT,
    Nutritional_deficiencies FLOAT,
    Meningitis FLOAT,
    Drowning FLOAT,
    Maternal_deaths FLOAT,
    Parkinson_disease FLOAT,
    Alcohol_disorders FLOAT,
    Intestinal_infectious_diseases FLOAT,
    Hepatitis FLOAT, 
    Fire FLOAT,
    Natural_disasters FLOAT,
    Conflict FLOAT,
    Terrorism FLOAT,
    FOREIGN KEY(Country)
    REFERENCES Countries(Country)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,

) AS SELECT
    country,
    year,
    Tuberculosis,
    Suicide,
    Drug_disorders,
    Cardiovascular_diseases, 
    Cancers,
    Respiratory_diseases, 
    Diabetes,
    Dementia,
    Lower_respiratory_infections,
    Neonatal_deaths,
    Diarrheal_diseases,
    Road_accidents,
    Liver_disease,
    Kidney_disease,
    Digestive_diseases,
    Malaria,
    Homicide,
    Nutritional_deficiencies,
    Meningitis,
    Drowning,
    Maternal_deaths,
    Parkinson_disease,
    Alcohol_disorders,
    Intestinal_infectious_diseases,
    Hepatitis, 
    Fire,
    Natural_disasters,
    Conflict,
    Terrorism
FROM CSVREAD('../data/processed/mortality_data.csv')

In [103]:
SELECT Mortality.country, Mortality.year, Mortality.Tuberculosis, S.OBS_VALUE FROM Mortality, Sanitation S
    WHERE Mortality.COUNTRY = 'Brazil' AND Mortality.year = S.year AND S.country = 'Brazil' AND S.INDICATOR = 'WS_PPL_S-ALB'

In [104]:
SELECT * FROM IDH

In [105]:
CREATE VIEW IDH2016 AS
    SELECT country, year, HDI_VALUE FROM IDH
        WHERE year = 2016

In [106]:
CREATE VIEW IDH1990 AS
    SELECT country, year, HDI_VALUE FROM IDH
        WHERE year = 1990

In [107]:
CREATE VIEW variacaoIDH AS
    SELECT IDH2016.country, IDH2016.HDI_VALUE/IDH1990.HDI_VALUE as variacao FROM IDH2016, IDH1990
        WHERE IDH2016.country = IDH1990.country

In [ ]:
SELECT country, variacao FROM variacaoIDH
    WHERE variacao = (SELECT MAX(variacao) FROM variacaoIDH)

In [109]:
SELECT * FROM variacaoIDH

In [110]:
CREATE VIEW TOPIDH AS
    SELECT country, HDI_value FROM IDH
    WHERE year = 2016
    ORDER BY HDI_value desc

In [111]:
CREATE VIEW TOP20IDH AS 
    SELECT top 20 * FROM TOPIDH

In [112]:
CREATE VIEW TOPIDHSUICIDE AS
    SELECT TOP20IDH.country, TOP20IDH.HDI_VALUE, M.Suicide FROM TOP20IDH, Mortality M
        WHERE TOP20IDH.country = M.country AND M.year = 2016

In [113]:
SELECT * FROM TOPIDHSUICIDE

In [114]:
SELECT AVG(Suicide) FROM Mortality
    WHERE year = 2016

1.3891157886018581

In [115]:
SELECT M.country, M.year, M.Drug_disorders, IDH.HDI_VALUE FROM Mortality M, IDH
    WHERE M.country = 'Brazil' AND IDH.country = 'Brazil' AND IDH.year = M.year

In [116]:
SELECT * FROM Mortality

In [117]:
SELECT * FROM Mortality
    WHERE country = 'Mozambique' AND (year = 1990 OR year = 2016)

In [118]:
CREATE VIEW DADOSSUICIDIO AS
    SELECT C.Region, M.year, AVG(M.Suicide) SuicidioMedio FROM Mortality M, Countries C
            WHERE M.country = C.country
            GROUP BY (C.Region, M.year)
            ORDER BY C.Region

In [124]:
SELECT * FROM DADOSSUICIDIO
 ORDER BY Region

In [123]:
SELECT D1.Region FROM DADOSSUICIDIO D1
    WHERE D1.year = 2016 AND D1.SUICIDIOMEDIO > (SELECT D2.SUICIDIOMEDIO FROM DADOSSUICIDIO D2 WHERE D2.year = 1990 AND D1.Region = D2.Region)